# Classical Adiabatic Cloud Structure

**References**
1. Chapter 7 Rogers and Yau: A short course in Cloud Physics
2. Robert Wood:Relationships between optical depth, liquid water path, droplet concentration and effective radius in an adiabatic layer cloud
2. 

In [11]:
import numpy as np
from metpy.units import units
from metpy.constants import *
import metpy.calc as mpcalc 

## Diffusional growth of single droplet


In [12]:
def mu(T):
    '''
    Compute the mu from the parameterization of mu(T) on pg 102 of R&Y book
    input:
    T: temperature in Kelvin
    P: pressure
    
    '''
    T_Kelvin = T.to('K').magnitude
    return 1.72*1e-5 *(393.0/(T_Kelvin+120.0))*np.power(T_Kelvin/273,3.0/2.0) *units('kg/m/s')

In [13]:
# define some constants
def K_diff(T):
    import metpy.interpolate as mpinterp
    '''
    compute the molecular diffusion coefficient "K" from the tabulated value 
    '''
    T_tab = np.arange(-40,40,10) * units('degreeC')
    K_tab = np.array([2.07,2.16,2.24,2.32,2.40,2.48,2.55,2.63])*1e-2*units('J/m/s/K')
    P0    = 100 * units('kPa')
    mu_tab = mu(T_tab)
    mu_in  = mu(T)
    return mpinterp.interpolate_1d(mu_in,mu_tab,K_tab)

def D_diff(T,P):
    import metpy.interpolate as mpinterp
    '''
    compute the Coefficient of thermal conductivity of air "D" from the tabulated value 
    '''
    P0    = 100 * units('kPa')
    T_tab = np.arange(-40,40,10) * units('degreeC')
    D_tab = np.array([1.62,1.76,1.91,2.06,2.21,2.36,2.52,2.69])*1e-5*units('m^2/s')
    mu_tab = mu(T_tab)
    mu_in  = mu(T)
    return mpinterp.interpolate_1d(mu_in*T,mu_tab*T_tab,D_tab) * P0/P



In [14]:
def Fk(T):
    '''
    compute the Fk term in Eq. 7.17 of R&T book
    
    '''
    c1=(water_heat_vaporization/water_gas_constant/T-1.0)
    c2= water_heat_vaporization*density_water/K_diff(T)/T
    return (c1*c2).to_base_units()

def Fd(T,P):
    '''
    compute the Fd term in Eq. 7.17 of R&T book
    
    '''
    es = mpcalc.saturation_vapor_pressure(T)
    return (density_water*water_gas_constant*T/D_diff(T,P)/es).to_base_units()

def constant_b(T,solute_ion_num,solute_mass,solute_mole_weight):
    '''
    Compute the constant b in Kohlor curve
    solute_ion_num:     ion number of solute (i.e., 2 for Nacl, Na+ Cl-)
    solute_mass:        mass of the solution (e.g., sulfate, salt) [kg]
    solute_mole_weight: molecular mass of solution [kg/mol]
    '''
    molecular_weight_ratio = water_molecular_weight/solute_mole_weight
    b = (3.0*solute_ion_num*water_molecular_weight*solute_mass) / \
        (4.0*np.pi*density_water*solute_mole_weight)                   #page 87 of RY
    return b.to_base_units()

def constant_a(T):
    '''
    Compute the constant a in Kohlor curve
    '''
    sigma_w =  0.075 *units('N/m')        #[N / m] air-water surface tension factor
    c2 = 2.0*sigma_w/(water_gas_constant*density_water)
    a = c2/T                         #page 88 of RY
    return a.to_base_units()
     

In [15]:
class Droplet():
    def __init__(self):
        pass
    def set_conditions(self,radius,temperature,pressure,\
                            solute_ion_num = 2,\
                            solute_mass=1e-12*units('g'),\
                            solute_mole_weight=58.44 * units('g/mol')):
        '''
        inputs:
        radius_um:     radius of the droplet in micron meter 
        temperature_K: temperature of the droplet in Kelvin
        pressure_kPa:  enviromental pressure in kPa    
        '''
        self.r = radius
        self.T = temperature
        self.P = pressure
        self.es = mpcalc.saturation_vapor_pressure(self.T)
        self.solute_ion_num = solute_ion_num 
        self.solute_mass =solute_mass
        self.solute_mole_weight = solute_mole_weight
        self.a = constant_a(self.T)
        self.b = constant_b(self.T,\
                            self.solute_ion_num,\
                            self.solute_mass,\
                            self.solute_mole_weight)

    def diffusion_growth_tendency(self,S):
        '''
        Compute the diffusional growth rate drdt based on Eq. 7.17 of R&T book
        inputs:
        S Supersaturation ratio
        '''
        self.Fk = Fk(self.T) #(c1*c2).to_base_units() #Fk constant based on Eq. (7.17) of R&Y book
        self.Fd = Fd(self.T,self.P) #(density_water*water_gas_constant*self.T/D_diff/self.es).to_base_units()
        self.xi1= 1.0/(self.Fk+self.Fd)
        self.xi = (S-1)*self.xi1 #(S-1-self.a/self.r+self.b/self.r**3)*self.xi1
        self.drdt = 1/self.r*self.xi #(1.0/self.r*((S-1.0-self.a/self.r+self.b/self.r**3)/(self.Fk+self.Fd))).to_base_units()
        #self.drdt = (1.0/self.r*((S-1.0)/(self.Fk+self.Fd))).to_base_units()

### examples of single droplet growth by diffsion in Table 7.2 from Mason 1971
The results are similar but a bit faster that thoses in Table 7.2. It seems that the a/r and b/r^3 terms are not considered in Mason 1971. 

In [16]:
def single_droplet_growth_example(SS,T,P,\
                                  initial_r,final_r,\
                                  dt=1e-1*units('s'),\
                                  solute_mass=1e-13*units('g')):
    '''
    this program computes the time needed to grow a single droplet
    from the initial radius 
    '''
    d  = Droplet()
    d.set_conditions(initial_r,T,P,solute_mass=solute_mass)
    S  = 1.0+SS
    t0 = 0
    while d.r<final_r:
        d.diffusion_growth_tendency(S)
        dr = d.drdt*dt
        d.set_conditions(d.r+dr,T,P)
        t0+=dt
    return t0

T  = 283*units('K')
P  = 80*units('kPa')
SS = 0.0005
r0  = 1.0*1e-6*units('m')
r1  = 3.0*1e-6*units('m')
time_to_grow = single_droplet_growth_example(SS,T,P,r0,r1,dt=1e-2*units('s'),solute_mass=1e-14*units('g'))
print(time_to_grow)
    

80.9400000000045 second


##  growth of droplet population

In [17]:
class air_parcel(Droplet):
    epsilon = water_gas_constant / dry_air_gas_constant
    def __init__(self):
        pass
    def init_conditions(self,Temperature,Pressure,Height,Vertical_velocity,Vapor_mass_mr):
        self.T=Temperature # temperature of air parcel
        self.P=Pressure    # pressure of air parcel
        self.H=Height      # height of air parcel   
        self.W=Vertical_velocity
        self.Qv=Vapor_mass_mr # mass mixing ratio of water vapor
        self.E =mpcalc.vapor_pressure(self.P,self.Qv).to('Pa')
        self.Es=mpcalc.saturation_vapor_pressure(self.T).to('Pa')
        self.S = self.E/self.Es
        self.Rho = mpcalc.density(self.P,self.T,self.Qv)
        self.compute_Q1()
        self.compute_Q2()
        
    def set_droplets(self,Droplet_Num,Droplet_Radius):
        self.Nc = Droplet_Num # number of the droplets corresponding to each droplet radius
        self.droplets=[]
        self.Ql=0.0 * units('kg/m^3')
        #print('1,LWC=',self.Ql)
        for i in range(len(Droplet_Radius)):
            drop = Droplet()
            drop.set_conditions(Droplet_Radius[i],self.T,self.P)
            self.droplets.append(drop)
            self.Ql+=density_water * 4.0/3.0*np.pi*Droplet_Radius[i]**3 * Droplet_Num[i]
            
    
    def compute_Q1(self):
        c1 = epsilon*water_heat_vaporization*earth_gravity/dry_air_gas_constant/dry_air_spec_heat_press/self.T
        c2 = earth_gravity/dry_air_gas_constant
        self.Q1=(1.0/self.T*(c1-c2)).to_base_units()
        
        
    def compute_Q2(self):
        
        c1 = dry_air_gas_constant*self.T/epsilon/self.Es
        c2 = epsilon * water_heat_vaporization**2/self.P/self.T/dry_air_spec_heat_press
        self.Q2 = (self.Rho*(c1+c2)).to_base_units()
        
    def adiabatic_ascend(self,dt=0.1*units('s')):
        dz = self.W*dt
        
        P_term = self.Q1*self.W
        drdt=[]
        old_r=[]
        new_r=[]
        old_Ql=self.Ql 
        for N in range(len(self.Nc)):
            self.droplets[N].diffusion_growth_tendency(self.S)
            drdt.append(self.droplets[N].drdt)
            old_r.append(self.droplets[N].r)
            new_r.append(self.droplets[N].r+drdt[-1]*dt)
        self.set_droplets(self.Nc,new_r)
        dQl = ((self.Ql-old_Ql)/self.Rho)[0]
        C_term = self.Q2*(dQl/dt)
        
        #print("P_term",P_term.to_base_units(),"C_term",C_term)
        
        self.dSdt = P_term - C_term
        dP    = - (self.Rho*earth_gravity*dz).to('hPa')
        #print('dP,dQl',dP,dQl)
        dq    = (-earth_gravity*dz + water_heat_vaporization*dQl)
        #print('dq',dq.to_base_units())
        dT    = (dq/dry_air_spec_heat_press).to('K')
        #print('dT',dT.to_base_units())
        dS    = self.dSdt * dt
        
        self.P +=  dP
        self.T +=  dT
        self.S +=  dS
        self.H +=  dz
        self.Es =  mpcalc.saturation_vapor_pressure(self.T).to('Pa')
        self.E  =  self.S * self.Es
        self.Qv =  mpcalc.mixing_ratio(self.E,self.P)
        self.Rho = mpcalc.density(self.P,self.T,self.Qv)    
        self.compute_Q1()
        self.compute_Q2()

### plot the figure 7.2 the Q1 and Q2 terms

In [19]:
T = 283.0 * units('K')
P = 900 * units('hPa')
W =  50 * units('cm/s')
mr = 9.0 *units('g/kg')
air = air_parcel()
air.init_conditions(T,P,1000*units('m'),W,mr)
CDNC=75
Nc_r = np.full(CDNC,1) * units('cm^-3')
drop_r = np.full(CDNC,0.5*1e-6) *units('m')
air.set_droplets(Nc_r,drop_r)
# print(air.droplets[30].r)
# print(air.Nc,air.Ql.to('g/m^3'),air.S)
print('before',air.S,air.P,air.T)
air.adiabatic_ascend(dt=1*units('s'))
print('after',air.S,air.P,air.T)

before 1.0566836945301203 dimensionless 900 hectopascal 283.0 kelvin
after 0.9417346698146685 dimensionless 899.9459687815679 hectopascal 283.99802784781366 kelvin
